# 1 Loading Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 2 Reading a part of the CSV

In [2]:
skiprows=10227384
nrows=1704564

In [3]:
test=pd.read_csv("../input/testing/test.csv.gzip",compression="gzip",skiprows=range(1,skiprows),nrows=nrows)

# 3 Understanding the Data

# 3.1 Shape of the Data

In [4]:
test.shape

(1136379, 170)

# 3.2 Find the Data Types

In [5]:
test.dtypes

customer_ID     object
S_2             object
P_2            float64
D_39             int64
B_1            float64
                ...   
D_140            int64
D_141          float64
D_143            int64
D_144          float64
D_145            int64
Length: 170, dtype: object

# 3.3 Glimps of Data

In [6]:
test.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,D_41,B_3,D_43,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,e669dd27dd84dfc5bd8ee32baf39a2ea89252657ef17ad...,2018-09-02,0.760419,9,0.029606,0.810839,0.0,0.0,0.003970,0.078262,...,-1,-1,-1,-1,0,0,0.0,0,0.009921,0
1,e669dd27dd84dfc5bd8ee32baf39a2ea89252657ef17ad...,2018-10-23,0.853384,0,0.041925,1.005659,0.0,0.0,0.011763,0.078262,...,-1,-1,-1,-1,0,0,0.0,0,0.006576,0
2,e669dd27dd84dfc5bd8ee32baf39a2ea89252657ef17ad...,2018-11-04,0.846441,10,0.062382,1.003084,0.0,0.0,0.009071,0.078262,...,-1,-1,-1,-1,0,0,0.0,0,0.003138,0
3,e669dd27dd84dfc5bd8ee32baf39a2ea89252657ef17ad...,2018-12-04,0.854945,11,0.105960,1.000066,0.0,0.0,0.003787,0.078262,...,-1,-1,-1,-1,0,0,0.0,0,0.001557,0
4,e669dd27dd84dfc5bd8ee32baf39a2ea89252657ef17ad...,2019-01-04,0.848144,0,0.012834,1.006813,0.0,0.0,0.004408,0.078262,...,-1,-1,-1,-1,0,0,0.0,0,0.009747,0


Thus we can reduce the size of the data to float16

# 4 Data Compression

In [7]:
for col in test.columns:
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col]/(test[col].max()-test[col].min())
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col].astype("float16")

# 5 Label encoding/factorizing the remaining character variables

The categorical features are the followings ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [8]:
columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 
for col in columns: 
    print(test[col].value_counts())

 0    961470
 1    164433
 2     10194
-1       282
Name: B_30, dtype: int64
 2    412646
 3    278316
 1    208135
 5     87714
 7     63844
 4     49625
 6     35817
-1       282
Name: B_38, dtype: int64
 1    696384
 0    411464
-1     28531
Name: D_114, dtype: int64
 0    1104401
-1      28531
 1       3447
Name: D_116, dtype: int64
 0    308562
 5    243201
 4    231095
 3    124772
 6     98549
 7     80825
-1     28531
 2     20844
Name: D_117, dtype: int64
 0    881450
 1    226398
-1     28531
Name: D_120, dtype: int64
 2    863914
 1    233226
 0     30999
-1      8240
Name: D_126, dtype: int64
3    841824
0    182936
4     92931
1     11124
2      4877
5      2687
Name: D_63, dtype: int64
 0    600856
 3    294472
 2    203076
-1     37975
Name: D_64, dtype: int64
-1    1002737
 1     133642
Name: D_66, dtype: int64
 6    562717
 5    262087
 4    102719
 3     98554
 2     48456
-1     34757
 1     27089
Name: D_68, dtype: int64


* D_64 has category 1 missing which is present in training data set.
* D_66 has category 0 missing which is present in training dataset.
* D_68 has category 0 missing which is present in training dataset.

In [9]:
test.groupby(by="customer_ID")

Here we factorize the date column.

In [10]:
test["S_2"]=test["S_2"].str.slice(start=0,stop=7)
mapping={"2018-04":13,"2018-05":14,"2018-06":15,"2018-07":16,"2018-08":17,"2018-09":18,"2018-10":19,"2018-11":20,"2018-12":21,"2019-01":22,"2019-02":23,"2019-03":24,"2019-04":25,"2019-05":26,"2019-06":27,"2019-07":28,"2019-08":29,"2019-09":30,"2019-10":31} 
test=test.replace({"S_2":mapping}) 
test["S_2"]

0          18
1          19
2          20
3          21
4          22
           ..
1136374    21
1136375    22
1136376    23
1136377    24
1136378    25
Name: S_2, Length: 1136379, dtype: int64

In [11]:
mapping={0:0.183402,1:0.627797,2:0.612930,-1:0.356151}
test=test.replace({"B_30":mapping}) 
test["B_30"]

0          0.183402
1          0.183402
2          0.183402
3          0.183402
4          0.183402
             ...   
1136374    0.183402
1136375    0.183402
1136376    0.183402
1136377    0.183402
1136378    0.183402
Name: B_30, Length: 1136379, dtype: float64

In [12]:
mapping={2:0.063306,3:0.328604,1:0.135264,5:0.559430,4:0.699716,7:0.498865,6:0.614916,-1:0.356151}
test=test.replace({"B_38":mapping}) 
test["B_38"]

0          0.063306
1          0.063306
2          0.063306
3          0.063306
4          0.063306
             ...   
1136374    0.135264
1136375    0.135264
1136376    0.135264
1136377    0.135264
1136378    0.135264
Name: B_38, Length: 1136379, dtype: float64

In [13]:
mapping={1:0.179497,0:0.345989,-1:0.437736}
test=test.replace({"D_114":mapping}) 
test["D_114"]

0          0.179497
1          0.179497
2          0.179497
3          0.179497
4          0.179497
             ...   
1136374    0.345989
1136375    0.345989
1136376    0.345989
1136377    0.345989
1136378    0.345989
Name: D_114, Length: 1136379, dtype: float64

In [14]:
mapping={0:0.242413,-1:0.437736,1:0.613341}
test=test.replace({"D_116":mapping}) 
test["D_116"]

0          0.242413
1          0.242413
2          0.242413
3          0.242413
4          0.242413
             ...   
1136374    0.242413
1136375    0.242413
1136376    0.242413
1136377    0.242413
1136378    0.242413
Name: D_116, Length: 1136379, dtype: float64

In [15]:
mapping={0:0.273270,4:0.269573,5:0.173700,3:0.340783,6:0.135087,7:0.134076,-1:0.437736,2:0.446632}
test=test.replace({"D_117":mapping}) 
test["D_117"]

0          0.173700
1          0.173700
2          0.173700
3          0.173700
4          0.173700
             ...   
1136374    0.134076
1136375    0.134076
1136376    0.134076
1136377    0.134076
1136378    0.134076
Name: D_117, Length: 1136379, dtype: float64

In [16]:
mapping={0:0.212713,1:0.471097,-1:0.437736}
test=test.replace({"D_120":mapping}) 
test["D_120"]

0          0.212713
1          0.212713
2          0.212713
3          0.212713
4          0.212713
             ...   
1136374    0.471097
1136375    0.471097
1136376    0.471097
1136377    0.471097
1136378    0.471097
Name: D_120, Length: 1136379, dtype: float64

In [17]:
mapping={2:0.232809,1:0.309875,0:0.209538,-1:0.468044}
test=test.replace({"D_126":mapping}) 
test["D_126"]

0          0.232809
1          0.232809
2          0.232809
3          0.232809
4          0.232809
             ...   
1136374    0.232809
1136375    0.232809
1136376    0.232809
1136377    0.232809
1136378    0.232809
Name: D_126, Length: 1136379, dtype: float64

In [18]:
mapping={3:0.262817,0:0.169865,4:0.290894,1:0.190918,2:0.237401,5:0.317444}
test=test.replace({"D_63":mapping}) 
test["D_63"]

0          0.169865
1          0.169865
2          0.169865
3          0.169865
4          0.169865
             ...   
1136374    0.262817
1136375    0.262817
1136376    0.262817
1136377    0.262817
1136378    0.262817
Name: D_63, Length: 1136379, dtype: float64

In [19]:
mapping={0:0.169255,3:0.343223,2:0.315242,-1:0.408642,1:0.220723}
test=test.replace({"D_64":mapping}) 
test["D_64"]

0          0.343223
1          0.343223
2          0.343223
3          0.343223
4          0.343223
             ...   
1136374    0.315242
1136375    0.315242
1136376    0.315242
1136377    0.315242
1136378    0.315242
Name: D_64, Length: 1136379, dtype: float64

In [20]:
mapping={-1:0.257171,1:0.183504,0:0.383747}
test=test.replace({"D_66":mapping}) 
test["D_66"]

0          0.257171
1          0.257171
2          0.257171
3          0.257171
4          0.257171
             ...   
1136374    0.257171
1136375    0.257171
1136376    0.257171
1136377    0.257171
1136378    0.257171
Name: D_66, Length: 1136379, dtype: float64

In [21]:
mapping={6:0.174952,5:0.262219,3:0.369780,4:0.343603,2:0.392902,-1:0.414493,1:0.403990,0:0.179717}
test=test.replace({"D_68":mapping}) 
test["D_68"]

0          0.174952
1          0.174952
2          0.174952
3          0.174952
4          0.174952
             ...   
1136374    0.174952
1136375    0.174952
1136376    0.174952
1136377    0.174952
1136378    0.174952
Name: D_68, Length: 1136379, dtype: float64

In [22]:
x=pd.DataFrame(np.linspace(start=12,stop=0,num=13))
y=pd.DataFrame([0.258934,0.257893,0.257176,0.256362,0.254698,0.252840,0.250988,0.248830,0.245882,0.242539,0.236346,0.238807,0.232017])

mapping={}
for i in range(13,32):
    mapping[i]=(x[0].corr(y[0])*y[0].std()/x[0].std())*(i-x[0].mean())+y[0].mean()
test=test.replace({"S_2":mapping}) 
test["S_2"]

0          0.275312
1          0.277528
2          0.279744
3          0.281961
4          0.284177
             ...   
1136374    0.281961
1136375    0.284177
1136376    0.286393
1136377    0.288609
1136378    0.290826
Name: S_2, Length: 1136379, dtype: float64

# 6 Normalise all the numerical columns

Here I normalize the numerical columns and change the data type to save some space. Here we have used min max normalizer.

In [23]:
for col in test.columns:
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col]/(test[col].max()-test[col].min())
    if col not in ['customer_ID', 'target']:
        test[col]=test[col].astype("float16")

# 7 Upload the test data

In [24]:
test.to_csv("test_1.csv.gzip",compression="gzip")